In [97]:
import pandas as pd
import unicodedata
import re
import hashlib

In [98]:
df = pd.read_csv(
    "../Data Layer/silver/Viagens_2025_LIMPAS.csv",
    sep=";",
    encoding="latin1",
    decimal=",",
    dayfirst=True,
    parse_dates=[
        'Período - Data de início',
        'Período - Data de fim'
    ]
)

In [99]:
df.columns = (
    df.columns
      .str.normalize('NFKD')
      .str.encode('ascii', errors='ignore')
      .str.decode('utf-8')
      .str.lower()
      .str.replace(r'[^\w]+', '_', regex=True)
      .str.strip('_')
)
df = df.rename(columns={
    'periodo_data_de_inicio': 'data_inicio',
    'periodo_data_de_fim': 'data_fim',
    'codigo_do_orgao_superior' : 'codigo_orgao_superior',
    'nome_do_orgao_superior' : 'nome_orgao_superior'
})

In [100]:
df['data_inicio'] = pd.to_datetime(df['data_inicio'])
df['data_fim'] = pd.to_datetime(df['data_fim'])

In [101]:
df['duracao_viagem_dias'] = (df['data_fim'] - df['data_inicio']).dt.days + 1

In [102]:
df['custo_medio_diario'] = (
    df['total_gasto']
    .where(df['duracao_viagem_dias'] > 0)
    / df['duracao_viagem_dias']
).round(2)

In [103]:
dim_tempo = (
    df[['data_inicio']]
    .drop_duplicates()
)

dim_tempo['tempo_id'] = range(1, len(dim_tempo) + 1)
dim_tempo['ano'] = dim_tempo['data_inicio'].dt.year
dim_tempo['mes_numero'] = dim_tempo['data_inicio'].dt.month
dim_tempo['mes_nome'] = dim_tempo['data_inicio'].dt.month_name()

In [104]:
dim_orgao_superior = (
    df[['codigo_orgao_superior', 'nome_orgao_superior']]
    .drop_duplicates()
)

dim_orgao_superior['orgao_superior_id'] = range(1, len(dim_orgao_superior) + 1)


In [105]:
dim_orgao_solicitante = (
    df[['codigo_orgao_solicitante', 'nome_orgao_solicitante', 'codigo_orgao_superior']]
    .drop_duplicates()
)

dim_orgao_solicitante['orgao_solicitante_id'] = range(1, len(dim_orgao_solicitante) + 1)


In [106]:
dim_viajante = (
    df[['cpf_viajante', 'nome', 'cargo', 'descricao_funcao']]
    .drop_duplicates()
)

dim_viajante['viajante_id'] = range(1, len(dim_viajante) + 1)


In [107]:
dim_motivo = (
    df[['motivo']]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_motivo['motivo_id'] = dim_motivo.index + 1
dim_motivo = dim_motivo[['motivo_id', 'motivo']]


In [108]:
fat_viagens = (
    df
    .merge(dim_tempo, on='data_inicio', how='left')
    .merge(dim_orgao_superior, on='codigo_orgao_superior', how='left')
    .merge(dim_orgao_solicitante, on='codigo_orgao_solicitante', how='left')
    .merge(dim_viajante, on='cpf_viajante', how='left')
    .merge(dim_motivo, on='motivo', how='left')
    [[
        'tempo_id',
        'orgao_superior_id',
        'orgao_solicitante_id',
        'viajante_id',
        'motivo_id',
        'valor_diarias',
        'valor_passagens',
        'valor_outros_gastos',
        'valor_devolucao',
        'total_gasto',
        'duracao_viagem_dias',
        'custo_medio_diario'
    ]]
)

In [109]:
colunas_hash = [
    'tempo_id',
    'orgao_superior_id',
    'orgao_solicitante_id',
    'viajante_id',
    'motivo_id',
    'total_gasto',
    'duracao_viagem_dias'
]

In [110]:
def gerar_hash_linha(row):
    texto = '|'.join(str(row[col]) for col in colunas_hash)
    return hashlib.sha256(texto.encode('utf-8')).hexdigest()

fat_viagens['fat_viagem_id'] = fat_viagens.apply(
    gerar_hash_linha,
    axis=1
)

In [111]:
fat_viagens = fat_viagens.reset_index(drop=True)
fat_viagens['fat_viagem_id'] = fat_viagens.index + 1

In [112]:
cols = ['fat_viagem_id'] + [c for c in fat_viagens.columns if c != 'fat_viagem_id']
fat_viagens = fat_viagens[cols]

In [113]:
fat_viagens.isna().sum()

fat_viagem_id           0
tempo_id                0
orgao_superior_id       0
orgao_solicitante_id    0
viajante_id             0
motivo_id               0
valor_diarias           0
valor_passagens         0
valor_outros_gastos     0
valor_devolucao         0
total_gasto             0
duracao_viagem_dias     0
custo_medio_diario      0
dtype: int64